In [1]:
!pip install feast

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!feast init my_project
%cd my_project/feature_repo

/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
05/17/2023 10:31:11 AM root INFO: creating /home/estudiante/repo/MLOPS/taller2/my_project/__pycache__
05/17/2023 10:31:11 AM root INFO: copying /home/estudiante/.local/lib/python3.10/s

In [3]:
import pandas as pd
pd.read_parquet("data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2023-05-02 10:00:00+00:00,1005,0.236377,0.972807,743,2023-05-17 10:31:11.416
1,2023-05-02 11:00:00+00:00,1005,0.158543,0.578608,170,2023-05-17 10:31:11.416
2,2023-05-02 12:00:00+00:00,1005,0.583431,0.024968,942,2023-05-17 10:31:11.416
3,2023-05-02 13:00:00+00:00,1005,0.175848,0.988660,713,2023-05-17 10:31:11.416
4,2023-05-02 14:00:00+00:00,1005,0.151660,0.912815,891,2023-05-17 10:31:11.416
...,...,...,...,...,...,...
1802,2023-05-17 08:00:00+00:00,1001,0.268602,0.306320,773,2023-05-17 10:31:11.416
1803,2023-05-17 09:00:00+00:00,1001,0.445904,0.679708,534,2023-05-17 10:31:11.416
1804,2021-04-12 07:00:00+00:00,1001,0.583974,0.082759,623,2023-05-17 10:31:11.416
1805,2023-05-09 22:00:00+00:00,1003,0.341435,0.665778,397,2023-05-17 10:31:11.416


In [4]:
!feast apply

/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/estudiante/.local/lib/python3.10/site-packages/feast/feature_store.py:561: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the function

In [5]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

store = FeatureStore(repo_path=".")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

print("----- Feature schema -----\n")
print(training_df.info())

print()
print("----- Example features -----\n")
print(training_df.head())

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   driver_id                           3 non-null      int64              
 1   event_timestamp                     3 non-null      datetime64[ns, UTC]
 2   label_driver_reported_satisfaction  3 non-null      int64              
 3   val_to_add                          3 non-null      int64              
 4   val_to_add_2                        3 non-null      int64              
 5   conv_rate                           3 non-null      float32            
 6   acc_rate                            3 non-null      float32            
 7   avg_daily_trips                     3 non-null      int32              
 8   conv_rate_plus_val1                 3 non-null      float64            
 9   conv_rate_plus_val2

In [6]:
entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

print("\n----- Example features -----\n")
print(training_df.head())


----- Example features -----

   driver_id                  event_timestamp  \
0       1002 2023-05-17 15:33:38.391382+00:00   
1       1001 2023-05-17 15:33:38.391382+00:00   
2       1003 2023-05-17 15:33:38.391382+00:00   

   label_driver_reported_satisfaction  val_to_add  val_to_add_2  conv_rate  \
0                                   5           2            20   0.061540   
1                                   1           1            10   0.445904   
2                                   3           3            30   0.038464   

   acc_rate  avg_daily_trips  conv_rate_plus_val1  conv_rate_plus_val2  
0  0.053379              670             2.061540            20.061540  
1  0.679708              534             1.445904            10.445904  
2  0.646903                2             3.038464            30.038464  


In [11]:
import os

current_time = os.popen('date -u +"%Y-%m-%dT%H:%M:%S"').read().strip()
os.environ['END_TS'] = '2023-05-17T23:59:59'

!feast materialize-incremental $current_time

/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
Materializing 2 feature views to 2023-05-17 10:39:27-05:00 into the sqlite online store.

driver_hourly_stats_fresh from 2023-05-16 15:39:29-05:00 to 2023-05-17 10:39:27-05:00:
100%|██

In [12]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

pprint(feature_vector)

{'acc_rate': [0.9526011943817139, 0.6637207269668579],
 'avg_daily_trips': [263, 222],
 'conv_rate': [0.15078987181186676, 0.487605482339859],
 'driver_id': [1004, 1005]}


In [1]:
%cd my_project

/home/estudiante/repo/MLOPS/taller2/my_project


In [4]:
!ls

feature_repo  __init__.py  README.md


In [5]:
%cd feature_repo

/home/estudiante/repo/MLOPS/taller2/my_project/feature_repo


In [ ]:
!feast ui

/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/estudiante/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:2804: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
INFO:     Started server process [2074223]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8888 (Press CT